# Jibo Teleop - Chat with GPT

## Step 1: import stuff

Run this only once

In [ ]:
import rclpy # ROS2
import time
import threading
from jibo_teleop_ros import JiboTeleop

rclpy.init()

## Step 2: Set your GPT config

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key="[your api key here!!]"
)
gpt_model = "gpt-3.5-turbo"

## Step 3: Chat with GPT 

In [ ]:
def main(args=None):
    teleop_connection = JiboTeleop()
    time.sleep(2)

    system_message =  "You are Jibo, a friendly social robot who likes to chat about different topics." \
                      "Pick a random topic to discuss and use personalized stories." \
                      "Always respond in one short fun and exciting sentence, " \
                      "and use language that is easy to understand for all ages." \
                      " Give turns to user to respond."
    # Spin in a separate thread
    thread = threading.Thread(target=rclpy.spin, args=(teleop_connection, ), daemon=True)
    thread.start()


    morning_name = 'Hello, am jibo! let me know when you want to chat about anything. '
    teleop_connection.send_tts_message(str(morning_name))
    time.sleep(2.0)
    teleop_connection.waitforJibo()

    chat_history=[
        {"role": "system", "content": system_message},
        {"role": "assistant", "content": morning_name}
    ]

    morning_motion = 'Misc/greetings_09.keys'
    teleop_connection.send_motion_message(morning_motion)
    time.sleep(2.0)
    teleop_connection.waitforJibo()

    while rclpy.ok():
        teleop_connection.JiboListen()
        time.sleep(2)
        teleop_connection.waitforJibo()

        if teleop_connection.asr_transcription != '' and teleop_connection.asr_transcription != 'NOSPEECH':

            chat_history.append({"role": "user", "name":"freind", "content": teleop_connection.asr_transcription})

            completion = client.chat.completions.create(
                model=gpt_model,
                messages=chat_history,
                )

            gpt_response = completion.choices[0].message.content
            chat_history.append({"role": "assistant", "content": gpt_response})

            teleop_connection.send_tts_message(str(gpt_response))
            time.sleep(2.0)
            teleop_connection.waitforJibo()


    teleop_connection.destroy_node()
    rclpy.shutdown()


if __name__ == '__main__':
    main()